In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import datetime
from group_lasso import GroupLasso
from sklearn.utils import resample, check_random_state
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
def standardize(X,y):
    # Standardize X to have mean = 0 std = 1
    # Standardize y to have mean = 0
    X_scaled = (X-np.mean(X,axis=0))/np.std(X,axis=0)
    y_scaled = y-np.mean(y)
    return X_scaled, y_scaled


def plot_coefficients(beta,alpha,alpha_opt=10,name=None):
    # Plotting regression coefficients vs lambda
    beta_opt = beta[:,np.argmin(np.abs(alpha-alpha_opt))]
    plt.figure()
    plt.plot(np.log10(alpha),beta.T,'-')
    plt.plot(np.log10(alpha_opt)*np.array([1,1]), [np.min(beta), np.max(beta)], 'k--')
    plt.xlabel(r'$\lambda$')
    plt.ylabel(r'$\beta$')
    plt.title(name)
    plt.show()

    
def plot_CV_MSE(alpha_vals, mse, alpha_opt, name=None):
    mse_mean = np.mean(mse,axis=1)
    mse_std = np.std(mse,axis=1)
    plt.figure()
    plt.errorbar(np.log10(alpha_vals), mse_mean, mse_std)
    plt.plot(np.log10([alpha_opt,alpha_opt]), [0,np.max(mse)],'k--')
    plt.xlabel(r'log($\lambda$)')
    plt.ylabel('MSE')
    plt.title(name)
    plt.show()
    print('Optimal value of lambda is: ', np.round(alpha_opt,3))

    
def Bootstrap_loop(X,y,model,b=20,N_samples=100):
    alphas = []
    betas = np.zeros((N_samples,X.shape[1]))
    for i in range(N_samples):
        index_vector = np.arange(len(y))
        boot_index = np.random.choice(index_vector, size=b)
        X_sample = X[boot_index,:]
        y_sample = y[boot_index]

        reg = model.fit(X_sample, y_sample)
        betas[i,:] = reg.coef_
    return betas

In [ ]:
df = pd.read_csv('energydata_complete.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

for i in range(len(df.columns)):
    plt.figure()
    plt.plot(df[df.columns[i]])
    plt.ylabel(df.columns[i])
    plt.show()

### Taking an n-hour mean

In [ ]:
df = df.resample('24h').mean()

### Generating extra features to describe time
weekday: number [0,6]\
weekstatus: binary describing weekend (1) or not (0)\
NSM: Number of Seconds from Midnight

These are used for filtering the data

In [ ]:
weekday = np.zeros(len(df))
weekstatus = np.zeros(len(df))
NSM = np.zeros(len(df))
month = np.zeros(len(df))

for i in range(len(df)):
    weekday[i] = df.index[i].weekday()
    weekstatus[i] = True if weekday[i] >= 5 else False  # False for workday, True for weekend
    NSM[i] = (df.index[i] - df.index[i].replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    month[i] = df.index[i].month

df['weekday'] = weekday
df['week status'] = weekstatus
df['NSM'] = NSM
df['month'] = month

In [ ]:
plt.figure()
plt.scatter(df['NSM'],df['Appliances'])
plt.xlabel('NSM')
plt.ylabel('Appliances')
plt.show()

### Filtering data and making training set
Example: Only february, after 16:00 and workday

In [ ]:
#df_train = df[(df.index.month==2) & (df['NSM']>=16*3600) & (df['NSM']<24*3600)]
df_train = df[(df.index.month==2)]
df_train = df_train.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

# Training data
y = np.array(df_train['Appliances']).reshape(-1,1)
X = np.array(df_train[df_train.columns[1:]])
X, y = standardize(X,y)

### Cross validation parameters

In [ ]:
# Creating array of penalties

n_alpha = 100 # Number of penalties
min_alpha = .01 # min penalty
max_alpha = 200 # Maximum penalty
alpha_vals = np.logspace(np.log10(min_alpha),np.log10(max_alpha),n_alpha)
alpha_vals = alpha_vals[::-1] # reversing array (some sklearn standard?)

### Ordinary least squares

In [ ]:
reg_lstsq = linear_model.LinearRegression(fit_intercept=False)
beta_ls = reg_lstsq.fit(X,y).coef_

### Lasso

In [ ]:
beta_lasso = np.zeros((X.shape[1],n_alpha))

cv_outs_lasso = []
min_cv = np.inf
for i in range(n_alpha):
    reg = linear_model.Lasso(alpha=alpha_vals[i], max_iter = 10000, fit_intercept = False)
    reg.fit(X,y)
    beta_lasso[:,i] = reg.coef_
    cv_out = cross_val_score(reg, X, y, cv=10, scoring='neg_mean_squared_error')
    cv_outs_lasso.append(cv_out)
    if -cv_out.mean() < min_cv:
        min_cv = -cv_out.mean()
        lasso_min_alpha = alpha_vals[i]
    
# Best model   
reg_lasso = linear_model.Lasso(alpha=lasso_min_alpha, max_iter = 10000, fit_intercept = False)

### Ridge

In [ ]:
beta_ridge = np.zeros((X.shape[1],n_alpha))
cv_outs_ridge = []
min_cv = np.inf
for i in range(n_alpha):
    reg = linear_model.ElasticNet(alpha=alpha_vals[i], max_iter = 10000, l1_ratio=0, fit_intercept = False)
    reg.fit(X,y)
    beta_ridge[:,i] = reg.coef_
    cv_out = cross_val_score(reg, X, y, cv=10, scoring='neg_mean_squared_error')
    cv_outs_ridge.append(cv_out)
    if -cv_out.mean() < min_cv:
        min_cv = -cv_out.mean()
        ridge_min_alpha = alpha_vals[i]
        
# Best model
reg_ridge = linear_model.ElasticNet(alpha=cv_ridge.alpha_, max_iter = 10000, l1_ratio=0.0, fit_intercept = False)

### Elastic net

In [ ]:
beta_elnet = np.zeros((X.shape[1],n_alpha))
cv_outs_elnet = []
min_cv = np.inf
for i in range(n_alpha):
    reg = linear_model.ElasticNet(alpha=alpha_vals[i], max_iter = 10000, l1_ratio=0.5, fit_intercept = False)
    reg.fit(X,y)
    beta_elnet[:,i] = reg.coef_
    cv_out = cross_val_score(reg, X, y, cv=10, scoring='neg_mean_squared_error')
    cv_outs_elnet.append(cv_out)
    if -cv_out.mean() < min_cv:
        min_cv = -cv_out.mean()
        elnet_min_alpha = alpha_vals[i]

# Best model
reg_elnet = linear_model.ElasticNet(alpha=elnet_min_alpha, max_iter = 10000, l1_ratio=0.5, fit_intercept = False)

### Group lasso

In [ ]:
beta_glasso = np.zeros((X.shape[1],n_alpha))
#group_keys= {"T":-1,
#            "RH":-1,
#            "":-1}
group_keys= {"T":1,
            "RH":2,
            "":-1}

groups = []
for var_name in df_train.columns[1:]:
    for key, value in group_keys.items():
        if key in var_name:
            groups.append(value)
            break
# Group the rooms, and outside together
groups=[-1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 10, 10, -1, -1]
print(*zip(groups, df_train.columns[1:]))
cv_outs = []
min_cv = np.inf
group_reg = 0.05
for i in range(n_alpha):
    reg = GroupLasso(
    groups=groups,
    group_reg=group_reg*alpha_vals[i],#alpha_vals[i],
    l1_reg=(1-group_reg)*alpha_vals[i],
    frobenius_lipschitz=True,
    #scale_reg="inverse_group_size",
    #scale_reg="inverse_group_size",
    subsampling_scheme=1,
    fit_intercept=False,
    random_state=0,
    supress_warning=True,
    n_iter=10000,
    tol=0.0001,
    )
    cv_out = cross_val_score(reg, X, y, cv=10, scoring='neg_mean_squared_error')
    cv_outs.append(cv_out)
    reg.fit(X, y)
    beta_glasso[:,i] = reg.coef_.reshape(-1,)
    #print(f"mean {cv_out.mean()}, stdev {cv_out.std()}")
    if -cv_out.mean() < min_cv:
        min_cv = -cv_out.mean()
        gl_min_alpha = alpha_vals[i]
        

In [ ]:
# Best model
reg_gl = GroupLasso(
    groups=groups,
    group_reg=group_reg*gl_min_alpha,#alpha_vals[i],
    l1_reg=(1-group_reg)*gl_min_alpha,
    frobenius_lipschitz=True,
    #scale_reg="inverse_group_size",
    #scale_reg="inverse_group_size",
    subsampling_scheme=1,
    fit_intercept=False,
    random_state=0,
    supress_warning=True,
    n_iter=10000,
    tol=0.0001,
    )

### Plotting coefficients vs lambda

In [ ]:
plot_coefficients(beta_ridge, alpha_vals, cv_ridge.alpha_, name='Ridge')
plot_coefficients(beta_lasso, alpha_vals, cv_lasso.alpha_, name='Lasso')
plot_coefficients(beta_elnet, alpha_vals, cv_elnet.alpha_, name='Elastic net')
plot_coefficients(beta_glasso,alpha_vals, gl_min_alpha,name='Grouped lasso')

In [ ]:
plot_CV_MSE(alpha_vals, -np.array(cv_outs_ridge), ridge_min_alpha, 'Ridge')
plot_CV_MSE(alpha_vals, -np.array(cv_outs_lasso), lasso_min_alpha, 'Lasso')
plot_CV_MSE(alpha_vals, -np.array(cv_outs_elnet), elnet_min_alpha, 'Elastic net')
plot_CV_MSE(alpha_vals, -np.array(cv_outs), gl_min_alpha, 'Grouped Lasso')

In [ ]:
data = {'Feature': list(df_train.columns[1:])}
df_results = pd.DataFrame(data)
df_results['Least squares'] = beta_ls.T
df_results['Lasso'] = beta_lasso[:,alpha_vals==cv_lasso.alpha_]
df_results['Ridge'] = beta_ridge[:,alpha_vals==cv_ridge.alpha_]
df_results['Elastic net'] = beta_elnet[:,alpha_vals==cv_elnet.alpha_]
df_results['Group Lasso'] = beta_glasso[:,alpha_vals==gl_min_alpha]
df_results

### Testing with another month

In [ ]:
'''

#df_test = df[(df.index.month==2) & (df['NSM']>=16*3600) & (df['NSM']<24*3600)]

months = [3,4]
regs = [reg_lstsq,reg_lasso,reg_ridge,reg_elnet,reg_glasso]
j = 0

fig, axs = plt.subplots(len(regs),len(months))
fig.set_figheight(15)
fig.set_figwidth(15)

for month in months:

    df_test = df[(df.index.month==month)]
    df_test = df_test.drop(['weekday', 'week status','month','NSM'], axis=1) # dropping the features used for filtering

    # Testing data
    y_t = np.array(df_test['Appliances']).reshape(-1,1)
    X_t = np.array(df_test[df_test.columns[1:]])
    X_t, y_t = standardize(X_t,y_t)
    y_p = reg_glasso.predict(X_t)

    i = 0
    for reg in regs:
        axs[i,j].plot([min(y_p),max(y_p)],[min(y_p),max(y_p)],'k--')
        axs[i,j].scatter(reg.predict(X_t),y_t)
        axs[i,j].grid()
        axs[i,j].axis('equal')
        print('Train score: ',reg.score(X,y))
        print('Test score: ',reg.score(X_t,y_t))
        i+=1
    j+=1
    
for ax in axs.flat:
    ax.set(xlabel='Predicted appliances', ylabel='True appliances')
for ax in axs.flat:
    ax.label_outer()
plt.show()
'''

### Bootstrap CV

In [ ]:
boot_samples = len(df_train)
boot_size = len(df_train)

betas_lasso = Bootstrap_loop(X,y,reg_lasso,b=boot_size,N_samples=boot_samples)
betas_ridge = Bootstrap_loop(X,y,reg_ridge,b=boot_size,N_samples=boot_samples)
betas_elnet = Bootstrap_loop(X,y,reg_elnet,b=boot_size,N_samples=boot_samples)
#betas_gl = Bootstrap_loop(X,y,reg_gl,b=boot_size,N_samples=boot_samples)

In [ ]:
plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_lasso,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Lasso')
plt.show()

plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_ridge,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Ridge')
plt.show()

plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.boxplot(betas_elnet,labels=df_train.columns[1:])
plt.xticks(rotation='vertical')
plt.title('Elastic Net')
plt.show()

In [ ]:
number_of_zeros = np.sum(betas_lasso == 0,axis=0)
y_pos = np.arange(len(df_train.columns[1:]))

# Sorting in descenting order
labels = df_train.columns[1:][number_of_zeros.argsort()][::-1]
number_of_zeros[::-1].sort()


plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.barh(y_pos,number_of_zeros ,align='center', alpha=0.5)
plt.yticks(y_pos, labels)
plt.show()